In [1]:
import os

os.chdir("../..")

In [2]:
from mars.db import collections
from mars.db.db_fields import LANGUAGE, CONTENT
from typing import List
from tqdm import tqdm
import tomotopy as tp

In [3]:
import spacy

nlp = spacy.load("en_core_web_sm")

nlp.disable_pipe("parser")
nlp.disable_pipe("ner")


def preprocess_text(text:str, user_data=None)->List[str]:
    doc = nlp(text)
    return [t.lemma_.lower() for t in doc if not t.is_stop if t.is_alpha]

2021-10-01 17:54:04.107020: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-01 17:54:04.107170: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
query = collections.segmented_texts.fetchByExample({LANGUAGE:"en"}, batchSize=1000)

texts = [doc[CONTENT] for doc in query]

corpus = tp.utils.Corpus(tokenizer = preprocess_text)
corpus.process(texts)
print("Starting model training...")
model = tp.CTModel(k=15, corpus=corpus)
model.train(10)
print("Model trained")
print(model.summary())

In [ ]:
model.summary()

Error: Kernel is dead

In [ ]:
extractor = tp.label.PMIExtractor(min_cf=10, min_df=5, max_len=5, max_cand=10000, normalized=True)
cands = extractor.extract(model)

labeler = tp.label.FoRelevance(model, cands, min_df=5, smoothing=1e-2, mu=0.25)
for k in range(model.k):
    print("== Topic #{} ==".format(k))
    print("Labels:", ', '.join(label for label, score in labeler.get_topic_labels(k, top_n=5)))
    for word, prob in model.get_topic_words(k, top_n=10):
        print(word, prob, sep='\t')
    print()
